In [1]:
import tensorflow as tf 
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models 
import tensorflow.keras.optimizers as optim 


import cv2
import numpy as np 
import matplotlib.pyplot as plt 
import random

import warnings 

warnings.filterwarnings("ignore")

train_images = []
test_images = []

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
 !unzip "/content/drive/MyDrive/archive 3.zip.zip"

Archive:  /content/drive/MyDrive/archive 3.zip.zip
replace 128x128/apparel/image0000.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [4]:
IMG = (128, 128, 3)

In [5]:
def ConvBlock(x, num, filter, alpha=0.1, kernel=3, moment=0.9):
  for i in range(num):
    x=layers.Conv2D(filter, kernel, padding="same" )(x)
    x=layers.BatchNormalization(momentum=moment)(x)
    x=layers.LeakyReLU(alpha)(x)
  return x

def operation(inp, num=6, filter=32, start_kernel=7, kernel=3, moment=0.9, alpha=0.1 , gf=2):
  x = ConvBlock(inp, 1, filter, alpha, kernel, moment)
  for _ in range(num):
      filter = filter*gf
      x = ConvBlock(x, num, filter)
  x = ConvBlock(x, 1, 3)
  return x

def Stegnography(inp1, inp2, num:list=[2,2,2], filter=64, start_kernel=7, kernel=3, moment=0.9, alpha=0.1, gf=2):
    inp1 = layers.Input(inp1)
    encoder = operation(inp1, num[0], filter, start_kernel, kernel, moment, alpha, gf)
    inp2 = layers.Input(inp2)
    x = layers.Concatenate()([inp2, encoder])
    intermediate = operation(x, num[1], filter, start_kernel, kernel, moment, alpha, gf)
    decoder =  operation(intermediate, num[2], filter, start_kernel, kernel, moment, alpha, gf)

    model = models.Model(inputs=[inp1, inp2], outputs=[intermediate, decoder], name="ma_chuda")
    return model

model = Stegnography(IMG,IMG)
model.summary() 





  

Model: "ma_chuda"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 128, 128, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                          

In [6]:
import os
path = r"/content/128x128/"
folders = os.listdir(path)
for folder in folders:
  if folder[-4:]!=".csv":
    print(len(os.listdir(path+folder)))

8144
3347
5387
3301
32226
10488
23382
33063
5831
2402
4957


In [7]:
import os
folders = os.listdir(path)
VAL = []
for folder in folders:
  if folder[-4:]!= ".csv":
    VALID = []
    files = os.listdir(path+folder)
    for file in files[1000:1400]:
      VALID.append(cv2.imread(path+folder+"/"+file)/255.0)
    VAL.append(VALID)

In [8]:
len(VAL[0])

400

In [9]:
import os
folders = os.listdir(path)
IMGS = []
for folder in folders:
  if folder[-4:]!=".csv":
    IMAGE = []
    files = os.listdir(path+folder)
    for file in files[:1000]:
      IMAGE.append(cv2.imread(path+folder+"/"+file)/255.0)
    IMGS.append(IMAGE)

In [10]:
import random

In [ ]:
IMGS = np.array(IMGS)
VAL = np.array(VAL)

In [ ]:
plt.imshow(IMGS[0][0])

In [ ]:
XIMG = []
XDUM = []
VIMG = []
VDUM = []

for _ in range(2500):
    folder1 = random.randint(0, 10)
    folder2 = random.randint(0,10)
    imp = random.randint(0, 800)
    dum = random.randint(0, 199)
    dumb = IMGS[folder1][imp]
    imp = IMGS[folder2][imp+dum]
  
    inpdum = dumb.reshape(IMG[0],IMG[1],IMG[2])
    inpsex = imp.reshape(IMG[0],IMG[1],IMG[2])
    XIMG.append(inpsex)
    XDUM.append(inpdum)

XDUM = np.array(XDUM)
XIMG = np.array(XIMG)

for _ in range(500):
    folder1 = random.randint(0, 10)
    folder2 = random.randint(0, 10)
    imp = random.randint(0, 300)
    dum = random.randint(0, 99)
    dumb = VAL[folder1][imp]
    imp = VAL[folder2][imp+dum]
    inputs = np.concatenate([dumb.reshape(IMG[0], IMG[1], IMG[2]), imp.reshape(IMG[0], IMG[1], IMG[2])], axis = 2)
    inpdum = dumb.reshape(IMG[0], IMG[1], IMG[2])
    inpsex = imp.reshape(IMG[0], IMG[1], IMG[2])
    VIMG.append(inpsex)
    VDUM.append(inpdum)

VDUM = np.array(VDUM)
VIMG = np.array(VIMG)


In [ ]:
for lr, epoch in zip([1e-6], [10]):
  model.compile(loss="mae", optimizer=optim.Adam(learning_rate=lr), metrics=["accuracy"])
  model.fit([XIMG, XDUM], [XDUM, XIMG], epochs=epoch, verbose=1, batch_size = 20, validation_data=[[VIMG, VDUM], [VDUM, VIMG]], validation_batch_size=20)

In [ ]:
# code for displaying multiple images in one figure
c=211
#import libraries
import cv2
from matplotlib import pyplot as plt

# create figure
fig = plt.figure(figsize=(10, 7))
out1, out2 = model([VDUM[c].reshape(-1, IMG[0], IMG[1], IMG[2]), VIMG[c].reshape(-1, IMG[0], IMG[1], IMG[2])])
# setting values to rows and column variables
rows = 2
columns = 2
fig.add_subplot(rows, columns, 1)

# showing image
plt.imshow(VIMG[c].reshape(IMG[0], IMG[1], IMG[2]))

plt.axis('off')
plt.title("dumb input")

# Adds a subplot at the 2nd position
fig.add_subplot(rows, columns, 2)

# showing image
plt.imshow(VDUM[c].reshape(IMG[0], IMG[1], IMG[2]))
plt.axis('off')
plt.title("input sercet")

# Adds a subplot at the 3rd position
fig.add_subplot(rows, columns, 3)

# showing image
plt.imshow(np.array(out1).reshape(IMG[0], IMG[1], IMG[2]))
plt.axis('off')
plt.title("Output Dumb")

# Adds a subplot at the 4th position
fig.add_subplot(rows, columns, 4)

# showing image
plt.imshow(np.array(out2).reshape(IMG[0], IMG[1], IMG[2]))
plt.axis('off')
plt.title("oitput secret")


In [ ]:
model.save("/content/drive/MyDrive/ujjwal128x128pro.h5")